In [1]:
from utility_dfplotter import *

# 1. Plot Nominal Distriubitons

In [4]:
def plotDataFrames():#
    for seletion in ["emu","mumu","mutau","ee","emu2","etau","e4j","mu4j","mu4j_fakes","e4j_fakes","mutau_fakes","etau_fakes"]:
    #for seletion in ["mumu"]:
        for nbjet in ["==1",">1"]:
            plotter = DFPlotter(seletion,nbjet)
            plotter.plotKinematics()
plotDataFrames()

# 2. Plot Systematics

In [3]:
for s in ["emu","mumu","mutau","mu4j","ee","emu2","etau","e4j"]:
    sp = SystematicPlotter(s,">=1",'FSR')
    sp.plotKinematics()

In [4]:
s = ['sd','df']

In [3]:
        selection = 'mumu0'
        Data = DFCutter(selection, '', "data2016").getDataFrame('')
#         MCzz = DFCutter(selection, '', "mcdiboson").getDataFrame('')
#         MCdy = DFCutter(selection, '', "mcdy").getDataFrame('')
#         MCt  = DFCutter(selection, '', "mct").getDataFrame('')
#         MCtt = DFCutter(selection, '', "mctt").getDataFrame('')

['/Users/zihengchen/Documents/Analysis/workplace/data/pickles/mumu0/data2016/ntuple_muon_2016H.pkl', '/Users/zihengchen/Documents/Analysis/workplace/data/pickles/mumu0/data2016/ntuple_muon_2016F.pkl', '/Users/zihengchen/Documents/Analysis/workplace/data/pickles/mumu0/data2016/ntuple_muon_2016G.pkl', '/Users/zihengchen/Documents/Analysis/workplace/data/pickles/mumu0/data2016/ntuple_muon_2016E.pkl', '/Users/zihengchen/Documents/Analysis/workplace/data/pickles/mumu0/data2016/ntuple_muon_2016D.pkl', '/Users/zihengchen/Documents/Analysis/workplace/data/pickles/mumu0/data2016/ntuple_muon_2016C.pkl', '/Users/zihengchen/Documents/Analysis/workplace/data/pickles/mumu0/data2016/ntuple_muon_2016B.pkl']


OSError: [Errno 22] Invalid argument

In [2]:
DFCutter(selection, '', "data2016")

NameError: name 'selection' is not defined

In [6]:
pd.read_pickle('/Users/zihengchen/Documents/Analysis/workplace/data/pickles/mumu0/data2016/ntuple_muon_2016H.pkl')

OSError: [Errno 22] Invalid argument